## Tutorial for the Intended Usage of the Feature Extraction Pipeline API

##### Please ensure that you have spaCy and the "es_core_news_md" pipeline installed

In [1]:
# !python -m spacy download es_core_news_md

In [2]:
import pprint
from utils import read_corpus
from features import feature_pipeline

# Setup pretty printer
p = pprint.PrettyPrinter(indent=4, width=140)

### Introduction
This notebook is intended to illustrate and explain the main workflows possible when using the feature extraction pipeline API for computing statistical/numerical features from a raw text in our collected corpus of Spanish texts. This notebook should be treated as a secondary resource for understanding the API, the primary resource being the file `features.py` containing the source code.

Let's begin by broadly describing the stages of the feature extraction pipeline. To calculate any statistical feature from a raw, unprocessed text, the following steps must occur in sequence:
1. Initialize the pipeline object
2. Pre-process (clean up) the text to remove stray whitespaces, numerals, characters, etc.
3. Extract fundamental attributes of the text (eg., tokens, POS tags, lemmas, etc.) using spaCy
4. Use the extracted attributes to calculate statistical features of the text (eg., total number of tokens, type-token ratio, pronoun density, etc.)

It is not possible to calculate a statistical feature without first extracting the fundamental attributes necessary for calculating that feature. In other words, we cannot reach stage 4 without completing stages 1, 2 and 3. However, the API is written with some shortcuts in place, allowing us to almost never need to explicitly call the `.preprocess()` method in our code. Depending on our usage we can even sometimes skip writing the spaCy step. It is most crucial to remember that the unprocessed text must be passed as an argument at some stage within the pipeline, but it is quite flexible as to which stage that should be.

Through this tutorial we will understand these shortcuts and learn how to best apply them in workflows.

### Corpus Reading
Let's begin by loading the corpus and seeing a broad view of what it looks like:

In [3]:
corpus = read_corpus()

for k, v in corpus.items():
    print(f"{k}: {len(v)}")

A1: 98
A2: 71
B: 152


Let's pick a text from the corpus for the purposes of this demo:

In [4]:
unprocessed_text = corpus["A1"][81]["content"]
print(unprocessed_text)

12. LOS DÍAS DE LA SEMANA
El año tiene cincuenta y dos semanas. Un
mes tiene cuatro semanas y dos o tres días más.
La semana tiene siete días. Los siete días se
llaman: Domingo, lunes, martes, miércoles,
jueves, viernes y sábado. El domingo es el
primer día. Es el día de reposo. El domingo
la gente no trabaja porque es el día de reposo.
Los otros seis días son días de trabajo. La
gente trabaja los otros días. Algunos discípulos
no están satisfechos con un día de reposo.
Ellos reposan también en la escuela. En los
Estados Unidos los discípulos van a la escuela
los lunes, los martes, los miércoles, los jueves
y los viernes. En España los discípulos van a
la escuela todos los días de trabajo; pero los
miércoles y los sábados ellos van solamente por
la mañana.




We can see that this piece of text is formatted somewhat "irregularly". It is a poem, so it has line breaks in the middle of grammatical sentences. It also has a title at the top, which is not a necessary component of the content of the text. In order to derive syntactic attributes like POS tags and dependency parses we will need to convert this text into a more standard form that can be interpreted by spaCy.

### Text Preprocessing
Let's create a pipeline for cleaning up this text and extracting important attributes and features from it.

In [5]:
# This step passes the un-processed text to the pipeline and automatically cleans it up using the .preprocess() method
pipe = feature_pipeline(unprocessed_text)
print(pipe.text)

los días de la semana el año tiene cincuenta y dos semanas. un mes tiene cuatro semanas y dos o tres días más. la semana tiene siete días. los siete días se llaman: domingo, lunes, martes, miércoles, jueves, viernes y sábado. el domingo es el primer día. es el día de reposo. el domingo la gente no trabaja porque es el día de reposo. los otros seis días son días de trabajo. la gente trabaja los otros días. algunos discípulos no están satisfechos con un día de reposo. ellos reposan también en la escuela. en los estados unidos los discípulos van a la escuela los lunes, los martes, los miércoles, los jueves y los viernes. en españa los discípulos van a la escuela todos los días de trabajo; pero los miércoles y los sábados ellos van solamente por la mañana.


The text looks much more standardized now, which makes it easier for downstream functions to extract things from it in a consistent manner.

In the above cell we only called the pipeline object constructor but it automatically gave us a cleaned text. That is because the constructor already contains a call for the `.preprocess()` method if a text has been supplied to the object. This behaviour is functionally equivalent to the following code:

In [6]:
pipe = feature_pipeline()
cleaned_text = pipe.preprocess(unprocessed_text)
print(cleaned_text)

los días de la semana el año tiene cincuenta y dos semanas. un mes tiene cuatro semanas y dos o tres días más. la semana tiene siete días. los siete días se llaman: domingo, lunes, martes, miércoles, jueves, viernes y sábado. el domingo es el primer día. es el día de reposo. el domingo la gente no trabaja porque es el día de reposo. los otros seis días son días de trabajo. la gente trabaja los otros días. algunos discípulos no están satisfechos con un día de reposo. ellos reposan también en la escuela. en los estados unidos los discípulos van a la escuela los lunes, los martes, los miércoles, los jueves y los viernes. en españa los discípulos van a la escuela todos los días de trabajo; pero los miércoles y los sábados ellos van solamente por la mañana.


\
As a rule of thumb, it is easiest to pass the unprocessed text to the pipeline at initialization, since the constructor has the effect of resetting all of the attributes to empty lists, giving the pipeline a blank slate for processing a new text that may come its way.

### Extracting Attributes of the Text using SpaCy
Now that we have cleaned up the text into a standard form, spaCy will be able to derive some fundamental syntactic attributes from the text. Some attributes of the text that we might be interested in are the sentences, tokens and POS tags. We can try accessing them, but they won't be accessible at this stage since by default the pipeline constructor does not execute any of the spaCy methods.

In [7]:
# The below calls to access class attributes will just return empty lists since those items have not been extracted yet
print(pipe.sentences)
print(pipe.tokens)
print(pipe.pos_tags)

[]
[]
[]


These attributes must be extracted from the text using spaCy's Spanish pipeline. It is recommended that we generate the attribute lists as and when we need them, since extracting all of the attributes for every text can be a bit slow (although the pre-processing of the text is typically the slowest step in the pipeline). We can extract some attributes as follows:

In [8]:
pipe.get_sentences()  # populates the pipe.sentences attribute
pipe.get_tokens()  # populates the pipe.tokens attribute

# Print out the attributes that were populated
p.pprint(pipe.sentences)
print()
print(pipe.tokens)

[   'los días de la semana el año tiene cincuenta y dos semanas.',
    'un mes tiene cuatro semanas y dos o tres días más.',
    'la semana tiene siete días.',
    'los siete días se llaman: domingo, lunes, martes, miércoles, jueves, viernes y sábado.',
    'el domingo es el primer día.',
    'es el día de reposo.',
    'el domingo la gente no trabaja porque es el día de reposo.',
    'los otros seis días son días de trabajo.',
    'la gente trabaja los otros días.',
    'algunos discípulos no están satisfechos con un día de reposo.',
    'ellos reposan también en la escuela.',
    'en los estados unidos los discípulos van a la escuela los lunes, los martes, los miércoles, los jueves y los viernes.',
    'en españa los discípulos van a la escuela todos los días de trabajo; pero los miércoles y los sábados ellos van solamente por la '
    'mañana.']

['los', 'días', 'de', 'la', 'semana', 'el', 'año', 'tiene', 'cincuenta', 'y', 'dos', 'semanas', '.', 'un', 'mes', 'tiene', 'cuatro', 'sema

\
Calling the methods above will populate the respective attributes with lists, but they also return the lists as outputs. We can assign the output to a variable and access it that way as well.

In [9]:
tags = pipe.get_pos_tags()
print(tags == pipe.pos_tags)  # check if they are the same
print(tags)

True
['DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'DET', 'NOUN', 'VERB', 'NUM', 'CCONJ', 'NUM', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'VERB', 'NUM', 'NOUN', 'CCONJ', 'NUM', 'CCONJ', 'NUM', 'NOUN', 'ADV', 'PUNCT', 'DET', 'NOUN', 'VERB', 'NUM', 'NOUN', 'PUNCT', 'DET', 'NUM', 'NOUN', 'PRON', 'AUX', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'PUNCT', 'NOUN', 'CCONJ', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'AUX', 'DET', 'ADJ', 'NOUN', 'PUNCT', 'AUX', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'DET', 'NOUN', 'ADV', 'VERB', 'SCONJ', 'AUX', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'DET', 'DET', 'NUM', 'NOUN', 'VERB', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'VERB', 'DET', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'ADV', 'VERB', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'PUNCT', 'PRON', 'AUX', 'ADV', 'ADP', 'DET', 'NOUN', 'PUNCT', 'ADP', 'DET', 'PROPN', 'PROPN', 'DET', 'NOUN', 'AUX', 'ADP', 'DET', 'NOUN', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN', 'PUNCT', 'DET',

Putting it all together, let's create a new pipeline object and give it the text to automatically pre-process. We can then just call a `.get_*` method to access the attribute using spaCy, completely eliminating the explicit writing of the pre-processing step.

In [10]:
pipe = feature_pipeline(unprocessed_text)
print(pipe.get_noun_chunks())

['los días', 'la semana', 'el año', 'semanas', 'un mes', 'semanas', 'días', 'la semana', 'días', 'los siete días', 'se', 'domingo', 'lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'el domingo', 'el primer día', 'el día', 'reposo', 'el domingo', 'la gente', 'el día', 'reposo', 'los otros seis días', 'días', 'trabajo', 'la gente', 'los otros días', 'algunos discípulos', 'un día', 'reposo', 'ellos', 'la escuela', 'los estados unidos', 'unidos', 'los discípulos', 'la escuela', 'los lunes', 'los martes', 'los miércoles', 'los jueves', 'los viernes', 'españa', 'los discípulos', 'la escuela', 'los días', 'trabajo', 'los miércoles', 'los sábados', 'ellos', 'la mañana']


Alternatively, we could also create a blank pipeline object and pass in the text through the `.get_*` method. This is functionally the same as the above. The only difference is that the text pre-processing will occur at the spaCy stage instead of the constructor stage.

In [11]:
pipe = feature_pipeline()
print(pipe.get_noun_chunks(unprocessed_text))

['los días', 'la semana', 'el año', 'semanas', 'un mes', 'semanas', 'días', 'la semana', 'días', 'los siete días', 'se', 'domingo', 'lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado', 'el domingo', 'el primer día', 'el día', 'reposo', 'el domingo', 'la gente', 'el día', 'reposo', 'los otros seis días', 'días', 'trabajo', 'la gente', 'los otros días', 'algunos discípulos', 'un día', 'reposo', 'ellos', 'la escuela', 'los estados unidos', 'unidos', 'los discípulos', 'la escuela', 'los lunes', 'los martes', 'los miércoles', 'los jueves', 'los viernes', 'españa', 'los discípulos', 'la escuela', 'los días', 'trabajo', 'los miércoles', 'los sábados', 'ellos', 'la mañana']


\
Here is a list of all of the spaCy features supported by our pipeline so far:
* sentences: \
    extraction function - `pipe.get_sentences()`, \
    attribute - `pipe.sentences`
* tokens: \
    extraction function - `pipe.get_tokens()`, \
    attribute - `pipe.tokens`
* lemmas: \
    extraction function - `pipe.get_lemmas()`, attribute - `pipe.lemmas`
* POS tags: \
    extraction function - `pipe.get_pos_tags()`, \
    attribute - `pipe.pos_tags`
* morphology tags: \
    extraction function - `pipe.get_morphology()`, \
    attribute - `pipe.morphs`
* dependency parses: \
    extraction function - `pipe.get_dependency_parses()`, \
    attribute - `pipe.parses`
* noun phrase chunks: \
    extraction function - `pipe.get_noun_chunks()`, \
    attribute - `pipe.noun_chunks`

What if we want to extract all of the spaCy features in one go, instead of calling each of the `.get_*` methods one by one? We can do that by calling the method `.full_spacy()` which will extract all of these features, OR we could initialize the pipeline object with the flag `full_spacy=True`.

In [12]:
pipe = feature_pipeline(unprocessed_text, full_spacy=True)
# Equivalent to:
# pipe = feature_pipeline()
# pipe.full_spacy(unprocessed_text)  # does not return any outputs, saves directly to attributes

# All of the following items will automatically be extracted as part of the spaCy pipeline:
print(pipe.text)
print()
print(pipe.parses)

# Commented out for brevity
# p.pprint(pipe.sentences)
# print(pipe.tokens)
# print(pipe.lemmas)
# print(pipe.pos_tags)
# print(pipe.morphs)
# print(pipe.noun_chunks)

los días de la semana el año tiene cincuenta y dos semanas. un mes tiene cuatro semanas y dos o tres días más. la semana tiene siete días. los siete días se llaman: domingo, lunes, martes, miércoles, jueves, viernes y sábado. el domingo es el primer día. es el día de reposo. el domingo la gente no trabaja porque es el día de reposo. los otros seis días son días de trabajo. la gente trabaja los otros días. algunos discípulos no están satisfechos con un día de reposo. ellos reposan también en la escuela. en los estados unidos los discípulos van a la escuela los lunes, los martes, los miércoles, los jueves y los viernes. en españa los discípulos van a la escuela todos los días de trabajo; pero los miércoles y los sábados ellos van solamente por la mañana.

['det', 'obl', 'case', 'det', 'nmod', 'det', 'obl', 'ROOT', 'nummod', 'cc', 'conj', 'obj', 'punct', 'det', 'nsubj', 'ROOT', 'nummod', 'obj', 'cc', 'nummod', 'cc', 'conj', 'conj', 'advmod', 'punct', 'det', 'obl', 'ROOT', 'nummod', 'obj',

### Numerical/Statistical Feature Extraction
The most important aspect of the feature extraction pipeline is the ability to derive statistical/numerical features from the text given to it. For a comprehensive guide of all of the features that this pipeline is capable of computing please see the project report (TODO: LINK TO PROJECT REPORT). \
\
Using the pipeline that we created and the attributes that we extracted in the previous cell, here is how we can derive some features from a text using the pipeline:

In [13]:
num_tokens = pipe.num_tokens()  # internally accesses pipe.tokens
log_op_density = pipe.logical_operators()  # internally accesses pipe.tokens
fh_score, syls_per_sent = pipe.fernandez_huerta_score()  # internally accesses pipe.tokens and pipe.sentences

print(num_tokens)
print(log_op_density)
print(fh_score)
print(syls_per_sent)

163
0.05161290322580645
88.93500000000002
19.46153846153846


It is important to note that any of the statistical feature functions can be called directly without needing to run any of the spaCy extractors first. As long as a feature pipeline object has been created, calling any of the feature functions will automatically extract the spaCy features necessary for computing the desired statistical feature. For example:

In [14]:
# Explicitly specifying full_spacy=False for demonstration purposes (default behaviour)
pipe = feature_pipeline(unprocessed_text, full_spacy=False)
# The above step simply cleans up the text. No spaCy features are extracted

# All of these methods try to access their required spaCy attributes, and if
# they find that the attribute does not yet exist the necessary method will
# be called interally to generate those attributes.

num_tokens = pipe.num_tokens()  # internally accesses pipe.tokens
log_op_density = pipe.logical_operators()  # internally accesses pipe.tokens
fh_score, syls_per_sent = pipe.fernandez_huerta_score()  # internally accesses pipe.tokens and pipe.sentences

print(num_tokens)
print(log_op_density)
print(fh_score)
print(syls_per_sent)

163
0.05161290322580645
88.93500000000002
19.46153846153846


Alternatively, please note that the order of where the text is provided to the pipeline may also be switched. That is, the unprocessed text does not have to be provided to the pipeline at the initialization stage; it can be provided directly to the feature function as well. The text will automatically be cleaned up and the necessary attributes will be extracted using spaCy, following which the statistic will be calculated.

In [15]:
pipe = feature_pipeline()

# Saves the cleaned text to pipe.text and the extracted list of tokens to pipe.tokens
num_tokens = pipe.num_tokens(text=unprocessed_text)
print(pipe.text)
print()
print(pipe.tokens)
print()

# No need to pass any arguments, it internally accesses pipe.tokens
log_op_density = pipe.logical_operators()

# No need to pass any arguments, it internally accesses pipe.tokens
# and extracts pipe.sentences from the cleaned up pipe.text
fh_score, syls_per_sent = pipe.fernandez_huerta_score()

print(num_tokens)
print(log_op_density)
print(fh_score)
print(syls_per_sent)

los días de la semana el año tiene cincuenta y dos semanas. un mes tiene cuatro semanas y dos o tres días más. la semana tiene siete días. los siete días se llaman: domingo, lunes, martes, miércoles, jueves, viernes y sábado. el domingo es el primer día. es el día de reposo. el domingo la gente no trabaja porque es el día de reposo. los otros seis días son días de trabajo. la gente trabaja los otros días. algunos discípulos no están satisfechos con un día de reposo. ellos reposan también en la escuela. en los estados unidos los discípulos van a la escuela los lunes, los martes, los miércoles, los jueves y los viernes. en españa los discípulos van a la escuela todos los días de trabajo; pero los miércoles y los sábados ellos van solamente por la mañana.

['los', 'días', 'de', 'la', 'semana', 'el', 'año', 'tiene', 'cincuenta', 'y', 'dos', 'semanas', '.', 'un', 'mes', 'tiene', 'cuatro', 'semanas', 'y', 'dos', 'o', 'tres', 'días', 'más', '.', 'la', 'semana', 'tiene', 'siete', 'días', '.', 

Finally, let's extract all of the available statistical features in one go. Accomplishing this is as simple as creating a pipeline object and calling the `.feature_extractor()` method. We explicitly only write two lines, and the unprocessed text can be supplied to the pipeline at either line, but under the hood all 4 stages of processing the text will take place. \
(If the object is initialized with a text, `.feature_extractor()` does not require any arguments. Otherwise, if the object is initialized *without* a text, `.feature_extractor()` must be given a text in order to perform pre-processing and spaCy attribute extraction.)

In [16]:
pipe = feature_pipeline(
    # These arguments are necessary for extracting the entire set of features
    dep_parse_flag=True,
    dep_parse_classpath="C:/Users/rsss9/stanza_corenlp/*",
    result_root="../wordnet_spa",
)
# It is necessary to start the CoreNLP client before extraction
pipe.corenlp_client.start()

2021-06-08 12:52:16 INFO: Using CoreNLP default properties for: spanish.  Make sure to have spanish models jar (available for download here: https://stanfordnlp.github.io/CoreNLP/) in CLASSPATH
2021-06-08 12:52:21 INFO: Starting server with command: java -Xmx5G -cp C:/Users/rsss9/stanza_corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 30000 -threads 5 -maxCharLength 100000 -quiet False -serverProperties spanish -annotators depparse -preload -outputFormat serialized


In [17]:
p.pprint(pipe.feature_extractor(unprocessed_text))

# ALTERNATIVELY:
# pipe = feature_pipeline(unprocessed_text)
# pipe.feature_extractor()

# It is necessary to stop the CoreNLP client after extraction
pipe.corenlp_client.stop()

{   'ADJ': 0.012269938650306749,
    'ADP': 0.07975460122699386,
    'ADV': 0.03067484662576687,
    'AUX': 0.049079754601226995,
    'CCONJ': 0.04294478527607362,
    'CONJ': 0.0,
    'CONTENT': 0.5571428571428572,
    'DET': 0.2147239263803681,
    'EOL': 0.0,
    'FUNCTION': 0.44285714285714284,
    'Fut': 0.0,
    'INTJ': 0.0,
    'Imp': 0.0,
    'NOUN': 0.294478527607362,
    'NUM': 0.049079754601226995,
    'PART': 0.0,
    'PRON': 0.018404907975460124,
    'PROPN': 0.018404907975460124,
    'PUNCT': 0.1411042944785276,
    'Past': 0.0,
    'Pres': 1.0,
    'SCONJ': 0.006134969325153374,
    'SPACE': 0.0,
    'SYM': 0.0,
    'VERB': 0.04294478527607362,
    'X': 0.0,
    'avg_ambiguation_all_words': 2.337078651685393,
    'avg_ambiguation_content_words': 2.757575757575758,
    'avg_degree_of_abstraction': 7.303819444444444,
    'avg_parse_tree_depth': 2.923076923076923,
    'avg_rank_of_lemmas_in_freq_list': 657.8036809815951,
    'avg_sent_length': 12.538461538461538,
    'ferna